# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [18]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [19]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'
file_path_list = []
# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    #print(root,dirs,files)
    
# join the file path and roots with the subdirectories using glob
    files = glob.glob(os.path.join(root,'*.csv'))
    for f in files:
        file_path_list.append(os.path.abspath(f))
    
print(len(file_path_list))
#print(file_path_list)

    

/home/workspace
39


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [20]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader) #skipping the header row in every file
        
 # extracting each data row one by one and append it        
        for line in csvreader:
           # print(line)
            full_data_rows_list.append(line)            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True) #skip initial spaces after delimiter

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''): #skipping row if there is no artist name
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


10082


In [21]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f)) #header

8560


In [22]:
file = file_path_list[0]
frame = pd.read_csv(file)
for file in file_path_list[1:]:
    df = pd.read_csv(file)
    frame = pd.concat([frame,df])


In [23]:
frame.shape

(10082, 17)

In [24]:
frame = frame[frame['artist'].notnull()]
frame.shape

(8559, 17)

In [25]:
frame.to_csv('event_datafile_new_1.csv', index = False)

# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [26]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [27]:
# TO-DO: Create a Keyspace called songs_database 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS songs_database 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [28]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('songs_database')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

select artist, songTitle , length from event_app where sessionId = 338 and itemInSession = 4;

### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [29]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
##"SELECT * from session_history WHERE sessionId = 338 AND itemInSession = 4"

query = "CREATE TABLE IF NOT EXISTS session_history (sessionId varint, itemInSession varint, artist_name text, \
song_title text, length float, PRIMARY KEY (sessionId, itemInSession))"

print(query)
session.execute(query)

       
        


CREATE TABLE IF NOT EXISTS session_history (sessionId varint, itemInSession varint, artist_name text, song_title text, length float, PRIMARY KEY (sessionId, itemInSession))


In [30]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_history (sessionId, itemInSession, artist_name, song_title, length)"
        query = query + "VALUES (%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0],line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [39]:

query = "SELECT * from session_history WHERE sessionId = 52 AND itemInSession = 2"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
df1 =pd.DataFrame(list(rows)) #converting results of query into dataframe
df1.head()
df1.columns = map(str.upper, df1.columns) #making column names UPPERCASE
df1.head()


#for row in rows:
#    print(row)
#    print(row.sessionid, row.iteminsession, row.artist_name, row.song_title, row.length)
    

,SESSIONID,ITEMINSESSION,ARTIST_NAME,LENGTH,SONG_TITLE
0,52,2,Mynt,166.94812,Playa Haters


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [40]:

## Query 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) \
## for userid = 10,  sessionid = 182 
query = "CREATE TABLE IF NOT EXISTS song_playlist_session (userid varint, sessionid varint, iteminsession varint,artist_name \
text,song_title text, listener_lastname text,listener_firstname text, PRIMARY KEY ((userId, sessionId), itemInSession))"

#itemInSession be the clustering column

print(query)
session.execute(query)
                    

                  

CREATE TABLE IF NOT EXISTS song_playlist_session (userid varint, sessionid varint, iteminsession varint,artist_name text,song_title text, listener_lastname text,listener_firstname text, PRIMARY KEY ((userId, sessionId), itemInSession))


In [41]:
## Query 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) \
## for userid = 10,  sessionid = 182 

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        #print(line)
        query = "INSERT INTO song_playlist_session (userid, sessionid, iteminsession, artist_name, song_title, \
        listener_lastname, listener_firstname)"
        query = query + "VALUES (%s,%s,%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0],line[9], line[4], line[1]))

In [42]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table

query = "select iteminsession, artist_name, song_title, listener_lastname, listener_firstname from song_playlist_session \
WHERE userid = 10 AND sessionid =182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.iteminsession,row.artist_name,row.song_title, row.listener_lastname,row.listener_firstname)

0 Down To The Bone Keep On Keepin' On Cruz Sylvie
1 Three Drives Greece 2000 Cruz Sylvie
2 Sebastien Tellier Kilometer Cruz Sylvie
3 Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Cruz Sylvie


In [52]:
## Query 3. Give me every user name (first and last) in my music app history who listened to the song "Uprising"
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query = "CREATE TABLE IF NOT EXISTS song_listener_history (song_title text, userid text, sessionid varint, \
iteminsession varint, listener_firstname text, listener_lastname text,  PRIMARY KEY (song_title, userid))"
print(query)
session.execute(query)

CREATE TABLE IF NOT EXISTS song_listener_history (song_title text, userid text, sessionid varint, iteminsession varint, listener_firstname text, listener_lastname text,  PRIMARY KEY (song_title, userid))


In [53]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        #print(line)
        query = "INSERT INTO song_listener_history (song_title, userid, sessionid, iteminsession, listener_firstname,\
        listener_lastname)"
        query = query + "VALUES (%s,%s,%s,%s,%s,%s)"
        session.execute(query, (line[9], line[10],int(line[8]), int(line[3]), line[1], line[4]))

In [54]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = "SELECT listener_firstname, listener_lastname from song_listener_history WHERE song_title='All Hands Against His Own'"
rows = session.execute(query)
for row_num in rows:
    print (row_num.listener_firstname, row_num.listener_lastname)


                    

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [21]:
## TO-DO: Drop the table before closing out the sessions
session.execute("drop table session_history")

In [22]:
session.execute("drop table song_playlist_session")

In [51]:
session.execute("drop table song_listener_history")

### Close the session and cluster connection¶

In [24]:
session.shutdown()
cluster.shutdown()